# A/B Test Review

In [184]:
import pandas as pd

## Load log file

In [185]:
df = pd.read_json('../website/logs/app.log', lines=True)

In [186]:
df.head(2)

,level,message,timestamp
0,info,"{'event': 'impression', 'experiments': [{'grou...",2024-04-08 06:36:20.314000+00:00
1,info,"{'event': 'click', 'experiments': [], 'extra':...",2024-04-08 06:36:21.918000+00:00


In [187]:
message_df = pd.json_normalize(df['message'])
message_df.head(2)

,event,experiments,page,url,username,extra.action,extra.elementId,extra.page,extra.productId,extra.paymentId
0,impression,"[{'group': 'test', 'name': 'SkipConfirmationSc...",,/,53e71b03-794d-4de9-8ae3-376980ee4ffb,NaN,NaN,NaN,NaN,NaN
1,click,[],welcome,/log,27b63d40-3661-4cf4-b531-0fc5a169b4e4,click,startButton,welcome,NaN,NaN


In [188]:
experiments_df = pd.json_normalize(df['message'], record_path='experiments')
experiments_df.head(2)

,group,name
0,test,SkipConfirmationScreen
1,test,SkipConfirmationScreen


In [189]:
df = pd.concat([df, message_df, experiments_df], axis=1).drop([
    'message', 
    'experiments', 
    'url',
    'level',
    'extra.elementId',
    'extra.productId',
    'extra.paymentId',
    'extra.action',
    'extra.page',
], axis=1)

In [190]:
df.head(100)

,timestamp,event,page,username,group,name
0,2024-04-08 06:36:20.314000+00:00,impression,,53e71b03-794d-4de9-8ae3-376980ee4ffb,test,SkipConfirmationScreen
1,2024-04-08 06:36:21.918000+00:00,click,welcome,27b63d40-3661-4cf4-b531-0fc5a169b4e4,test,SkipConfirmationScreen
2,2024-04-08 06:36:21.921000+00:00,impression,products,27b63d40-3661-4cf4-b531-0fc5a169b4e4,test,SkipConfirmationScreen
3,2024-04-08 06:36:23.532000+00:00,click,products,27b63d40-3661-4cf4-b531-0fc5a169b4e4,test,SkipConfirmationScreen
4,2024-04-08 06:36:23.538000+00:00,impression,payment,27b63d40-3661-4cf4-b531-0fc5a169b4e4,test,SkipConfirmationScreen
5,2024-04-08 06:36:23.545000+00:00,impression,payment,27b63d40-3661-4cf4-b531-0fc5a169b4e4,test,SkipConfirmationScreen
6,2024-04-08 06:36:25.382000+00:00,click,payment,27b63d40-3661-4cf4-b531-0fc5a169b4e4,NaN,NaN
7,2024-04-08 06:36:25.388000+00:00,impression,receipt,27b63d40-3661-4cf4-b531-0fc5a169b4e4,NaN,NaN


## Are test and control groups balanced?

In [191]:
df.groupby(['name', 'group'])['username'].nunique().reset_index(name='unique_count')

,name,group,unique_count
0,SkipConfirmationScreen,test,2


## How many people complete flow?